In [1]:
from pytezos import ContractInterface, pytezos, MichelsonRuntimeError
from decimal import Decimal

# alternative load from edo-net:
# contract = pytezos.contract('KT1HbiznMedpC5BcQnUCUd5zCaXcemsU25Sk')

In [2]:
contract_fn = 'crystal_ball_sasha.tz'
participants = dict(
    a='tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ',
    b='tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos',
    c='tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE'
)
contract = ContractInterface.from_file(contract_fn)

In [3]:
import time
from settings import settings

# it is not important to use pytezos.now():
run_time = int(time.time())
one_hour = 60*60

storage = settings['DEFAULT_INITIAL_CONTRACT_STORAGE']
storage.update({
    'targetRate': 6_000_000,
    'targetTime': run_time + one_hour,
})

### Полный тест (пока без unittest, чисто на ассёртах):

In [19]:
"""
test 1:
targetRate is 6.0 tez and real rate is 6.20 tez, so betFor pool wins.
three participants:
- participant A locks 100_000 utz betAgainst 
- participant B locks 200_000 utz betFor
- participant C locks 200_000 utz betFor
B and C win against A
participant B can withdraw 250_000
participant C can withdraw 250_000
"""

# participant A betAgainst with 100_000 utz:
res = contract.betAgainst().with_amount(100_000).interpret(
    storage=storage, sender=participants['a'])

assert res.operations == []

# participant B betFor with 200_000 utz:
res = contract.betFor().with_amount(200_000).interpret(
    storage=res.storage, sender=participants['b'])

# participant C betFor with 200_000 utz:
res = contract.betFor().with_amount(200_000).interpret(
    storage=res.storage, sender=participants['c'])

assert res.storage['betsForSum'] == 400_000
assert res.storage['betsAgainstSum'] == 100_000
assert len(res.storage['betsAgainstLedger']) == 1
assert len(res.storage['betsForLedger']) == 2

# calling close, should create opearaton with call to oracle get
res = contract.close().interpret(storage=res.storage, sender=participants['b'])
assert len(res.operations) == 1
operation = res.operations[0]
assert operation['destination'] == storage['oracleAddress']
assert operation['parameters']['entrypoint'] == 'get'
assert operation['parameters']['value']['args'][0]['string'] == storage['currencyPair']

# emulating close callback from oracle:
# 2 hours late:
callback_values = {
    'currencyPair': storage['currencyPair'],
    'lastUpdate': pytezos.now() + 2*one_hour,
    'rate': 6_200_000
}

res = contract.closeCallback(callback_values).interpret(
    storage=res.storage, sender=storage['oracleAddress'])
assert res.operations == []
assert res.storage['closedRate'] == callback_values['rate']
assert res.storage['isClosed']
assert res.storage['isBetsForWin']

# withrawing:
'''
# TODO: failwith: Participant is not win:
contract.withdraw().interpret(
    storage=res.storage, sender=participants['a'])
'''

res = contract.withdraw().interpret(
    storage=res.storage, sender=participants['b'])

assert len(res.operations) == 1
operation = res.operations[0]
assert operation['destination'] == participants['b']
assert int(operation['amount']) == 250_000
# assert that participant is removed:
assert res.storage['betsForLedger'][participants['b']] is None

def remove_none_records(dct):
    return {key: value for key, value in dct.items() if value is not None}

# need to remove None records before using this storage in next transaction:
res.storage.update({
    'betsForLedger': remove_none_records(res.storage['betsForLedger'])
})

res = contract.withdraw().interpret(
    storage=res.storage, sender=participants['c'])
operation = res.operations[0]

assert int(operation['amount']) == 250_000
assert res.storage['betsForLedger'][participants['c']] is None

In [235]:
# test 2: participant A makes two bets in a row:
# storage should contain sum of the bets


# test 3: participant tries to run callback: error should be raised

# test 4: participant tries to withdraw twice -> fail

# test 5: participant lose and tries to withdraw -> fail